In [1]:
import toml
from torch.utils import data

from tools.celeba import CelebALoader
from dataset import Split

from models.models import create_model
from tools.toml import load_option
from tools.mask import mask_iter

In [2]:
header_opt = load_option('options/header.toml')
loader = CelebALoader(header_opt.data_root)
dataset = Split(loader, header_opt.fine_size)
ty = ["attr", "identity", "bbox", "landmarks"]
valset = dataset.valid(ty)
valset

Dataset VisionDataset
    Number of datapoints: 19867
    Root keys: dict_keys(['splits', 'identity', 'bbox', 'landmarks_align', 'attr', 'Z'])
    Target type: ['attr', 'identity', 'bbox', 'landmarks']
    Split: valid
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [3]:
val_data = dataset.valid('bbox', header_opt.fine_size)
valset = data.DataLoader(val_data,
                           batch_size=header_opt.batch_size,
                           shuffle=True)
maskset = mask_iter(header_opt.mask_root, header_opt.fine_size)
print('训练集数量：', len(valset))
print('掩码数量：', len(maskset))

训练集数量： 19867
掩码数量： 12000


In [4]:
import time
import torch
import torchvision
model_opt = load_option('options/train2.toml')

model = create_model(model_opt)

csa_net
initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
---------- Networks initialized -------------
UnetGeneratorCSA(
  (model): UnetSkipConnectionBlock_3(
    (model): Sequential(
      (0): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): UnetSkipConnectionBlock_3(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2, inplace=True)
          (1): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(3, 3), dilation=(2, 2))
          (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (3): LeakyReLU(negative_slope=0.2, inplace=True)
          (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (6): UnetSkipConnectionBlock_3(
            (model): Sequential(
              (0): LeakyRe

In [ ]:
load_epoch=30
model.load(load_epoch)

In [5]:
import time
from pathlib import Path

In [ ]:
save_dir = '../measure/true'
save_dir = Path(save_dir)
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)

epoch = 1
i = 0
for image, mask in (valset):
    iter_start_time = time.time()
    image = image.cuda()
    mask = mask.cuda()
    mask = mask[0][0]
    mask = torch.unsqueeze(mask, 0)
    mask = torch.unsqueeze(mask, 1)
    mask = mask.byte()

    model.set_input(image, mask)
    model.set_gt_latent()
    model.test()
    real_A, real_B, fake_B = model.get_current_visuals()
    pic = (torch.cat([real_A, real_B, fake_B], dim=0) + 1) / 2.0
    torchvision.utils.save_image(pic, '%s/Epoch_(%d)_(%dof%d).jpg' % (
        save_dir, epoch, total_steps + 1, len(dataset_test)), nrow=1)